In [1]:
import numpy as np
import scipy as sp
from algorithms import QuboTriU, get_A, greedy
import joblib
import networkx as nx
import utils
import json
from glob import glob 
import os
import pandas as pd

In [2]:
df_optimum = pd.read_html("solutions.html")[0]
df_optimum.drop(df_optimum.index[0], inplace=True)
df_optimum.columns = [40, '40_optimum', 80, '80_optimum', 100, '100_optimum',
                120, '120_optimum', 140, '140_optimum', 160, '160_optimum']
df_optimum

,40,40_optimum,80,80_optimum,100,100_optimum,120,120_optimum,140,140_optimum,160,160_optimum
1,kcluster40_025_10_1,29,kcluster80_025_20_1,94,kcluster100_025_25_1,139,kcluster120_025_30_1,198,kcluster140_025_35_1,259,kcluster160_025_40_1,338
2,kcluster40_025_10_2,30,kcluster80_025_20_2,93,kcluster100_025_25_2,138,kcluster120_025_30_2,199,kcluster140_025_35_2,263,kcluster160_025_40_2,338
3,kcluster40_025_10_3,27,kcluster80_025_20_3,100,kcluster100_025_25_3,142,kcluster120_025_30_3,197,kcluster140_025_35_3,257,kcluster160_025_40_3,333
4,kcluster40_025_10_4,27,kcluster80_025_20_4,96,kcluster100_025_25_4,140,kcluster120_025_30_4,198,kcluster140_025_35_4,261,kcluster160_025_40_4,333
5,kcluster40_025_10_5,26,kcluster80_025_20_5,97,kcluster100_025_25_5,145,kcluster120_025_30_5,199,kcluster140_025_35_5,266,kcluster160_025_40_5,328
6,kcluster40_025_20_1,77,kcluster80_025_40_1,280,kcluster100_025_50_1,417,kcluster120_025_60_1,600,kcluster140_025_70_1,806,kcluster160_025_80_1,1047
7,kcluster40_025_20_2,84,kcluster80_025_40_2,273,kcluster100_025_50_2,417,kcluster120_025_60_2,611,kcluster140_025_70_2,805,kcluster160_025_80_2,1042
8,kcluster40_025_20_3,76,kcluster80_025_40_3,285,kcluster100_025_50_3,429,kcluster120_025_60_3,598,kcluster140_025_70_3,792,kcluster160_025_80_3,1026
9,kcluster40_025_20_4,75,kcluster80_025_40_4,284,kcluster100_025_50_4,413,kcluster120_025_60_4,600,kcluster140_025_70_4,795,kcluster160_025_80_4,1033
10,kcluster40_025_20_5,73,kcluster80_025_40_5,292,kcluster100_025_50_5,435,kcluster120_025_60_5,604,kcluster140_025_70_5,816,kcluster160_025_80_5,1021


We are generating qubos for the benchmark data to calculate the DkS using the quadratic penalty approach
 - g = (read A)
 - g = complement(g)  (because, we work with SkS)
 - $\frac{\mu}{2} > \min(\tilde{m}, k-1)$   where $\tilde{m}$ is no. of edges with heuristic (Greedy)
 - $\mu = 2\min(\tilde{m}, k-1) + 1$
 - Q = $\frac{1}{2} x^T A x + \frac{\mu}{2} (e^T x - k)^2$
 - Since BiqBin only works with Q containing integer elements, we consider Q = 2Q
  

In [20]:
def get_nearby_mus(mu):
    mus = []
    mu_max = mu + 2
    mu_min = 1

    mus = list(range(mu_max, mu_min, -10))
    mus.extend([mu, mu_min])
    mus = sorted(set(mus))
    
    # for i in range(mu_max, mu_min, -2):
    #     # if (mu+i <= 0):
    #     #     break
    #     # mus.append(mu+i)
    #     mus.append(i)
    return mus

In [22]:
sizes = [80]

for size in sizes:

    qubo_folder = f"qubo_folder/{size}"
    os.makedirs(qubo_folder, exist_ok=True)

    input_data_folder = f"k-cluster_{size}"
    files = os.listdir(input_data_folder)
    files = sorted([file for file in files if "_1.dat" in file])

    for file in files:
        instance_name = file.removesuffix(".dat")
        opt_val = int(df_optimum[df_optimum[size] == instance_name][f'{size}_optimum'].values[0])

        A_raw, A, k = get_A(f"{input_data_folder}/{file}")
        g = nx.from_numpy_array(A)
        g = nx.complement(g)

        g_greedy, dmax, dvag = greedy(g=g, k=k)
        mu_lb = 2*min(g_greedy.number_of_edges(), k-1) + 1
        mu_lb = int(mu_lb)
        
        print(f"{mu_lb = }")

        mus = get_nearby_mus(mu=mu_lb)
        max_mu = max(mus)
        width = len(str(max_mu))

        os.makedirs(f"{qubo_folder}/{instance_name}", exist_ok=True)

        for mu in mus:
            qubo_obj = QuboTriU(g=g)
            Q = qubo_obj.update(l=0, mu = mu, k=k)
            Q = 2*np.array(Q)
            data = utils.qubo_to_biqbin_representation(Q)
            data["instance"] = instance_name
            data["mu_lb"] = mu_lb
            data["mu"] = mu
            data["optimum"] = opt_val

            qubo_file_name = f"{instance_name}_mulb_{mu_lb:0{width}d}_mu_{mu:0{width}d}.json"
            with open(f"{qubo_folder}/{instance_name}/{qubo_file_name}", "w") as f:
                json.dump(data, f)


mu_lb = 39
mu_lb = 79
mu_lb = 119
mu_lb = 39
mu_lb = 79
mu_lb = 119
mu_lb = 27
mu_lb = 79
mu_lb = 119


In [23]:
sizes = [80]

for size in sizes:
    for instance_name in sorted(os.listdir(f"qubo_folder/{size}")):
        output_folder = f"outputs/{size}/{instance_name}/"
        os.makedirs(output_folder, exist_ok=True)
        for file in sorted(glob(f"qubo_folder/{size}/{instance_name}/*")):

            print(f"mpirun python3 biqbin_qubo.py {file} params {output_folder}")

mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_01.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_11.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_21.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_31.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_39.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_20_1/kcluster80_025_20_1_mulb_39_mu_41.json params outputs/80/kcluster80_025_20_1/
mpirun python3 biqbin_qubo.py qubo_folder/80/kcluster80_025_40_1/kcluster80_025_40_1_mulb_79_mu_01.json params outputs/80/kcluster80_025_40_1/